# Title: Cleaning Chicago Food Inspection Data.
#### All work is done WITHOUT use of packages such as Pandas or Numpy

# Background 

#### This Notebook was inspired by David Beazley's PyData 2016 talk.
#### The video lecture can be found here: https://www.youtube.com/watch?v=j6VSAsKAj98
#### The data is available at: https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5 
#### It consists of food inspection records for various premises in Chicago in the United States.
#### The aim of this notebook is to demonstrate cleaning operations carried out on the data. The Python Standard Library was used.

   # Import & explore the data.

In [1]:
import csv
food_inspection = list(csv.DictReader(open('Food_Inspections.csv'))) # This returns a List of Dictionaries.

In [2]:
def spaces(value,sepr = ".", dist = 20): # This is a function for spacing values so as to make output readible.
    dots = sepr*(dist-len(value))
    return dots

#### Data Structure

In [3]:
row_0 = food_inspection[0]
print('Python Datatypes',spaces('Python Datatypes'), 'Number of Entries')
print(type(food_inspection),spaces(str(type(food_inspection))), len(food_inspection),'instances')
print(type(row_0),spaces(str(type(row_0))),len(row_0),'columns')

Python Datatypes .... Number of Entries
<class 'list'> ...... 176217 instances
<class 'collections.OrderedDict'>  17 columns


#### Sample Instance

In [4]:
for x in row_0:
    print(x, spaces(x), row_0[x])

Inspection ID ....... 2229599
DBA Name ............ FRESHII
AKA Name ............ FRESHII
License # ........... 2309263
Facility Type ....... Restaurant
Risk ................ Risk 1 (High)
Address ............. 50 E WASHINGTON ST 
City ................ CHICAGO
State ............... IL
Zip ................. 60602
Inspection Date ..... 10/11/2018
Inspection Type ..... Canvass
Results ............. Out of Business
Violations .......... 
Latitude ............ 41.88333090689532
Longitude ........... -87.62603978976665
Location ............ (41.88333090689532, -87.62603978976665)


#### Columns & Datatypes:
##### All Columns are Strings.

In [5]:
def list_of_columns(dictionary):
    columns_list = [key for key in dictionary]
    return(columns_list)

def column_datatype(dictionary,column):
    datatype = type(dictionary[column])
    return(datatype)

column_list = list_of_columns(row_0)
print('Column',spaces('Column'),'Datatype\n')
for column in column_list:
    print(column, spaces(column), column_datatype(row_0,column))

Column .............. Datatype

Inspection ID ....... <class 'str'>
DBA Name ............ <class 'str'>
AKA Name ............ <class 'str'>
License # ........... <class 'str'>
Facility Type ....... <class 'str'>
Risk ................ <class 'str'>
Address ............. <class 'str'>
City ................ <class 'str'>
State ............... <class 'str'>
Zip ................. <class 'str'>
Inspection Date ..... <class 'str'>
Inspection Type ..... <class 'str'>
Results ............. <class 'str'>
Violations .......... <class 'str'>
Latitude ............ <class 'str'>
Longitude ........... <class 'str'>
Location ............ <class 'str'>


##### Functions for:  Set of distinct values in a column.
##### Count  of distinct values in a column.

In [6]:
def set_of_distinct_values_by_column(list_dictionaries, column):
    return( {dictionary[column] for dictionary in list_dictionaries} )

def count_distinct_values_by_column(list_dictionaries, column):
    return(len(set_of_distinct_values_by_column(list_dictionaries, column)))

In [7]:
counts = [count_distinct_values_by_column(food_inspection,column) for column in column_list]
list_distinct_values_and_count = [(x,y) for x,y in zip(column_list, counts)]#
print('Column', spaces('Column'), 'Number of Distinct Values\n')
list_distinct_values_and_count.sort(key=lambda x: x[1], reverse=True) # Sort the list by count
for column in list_distinct_values_and_count:
    print(column[0], spaces(column[0]),column[1])

Column .............. Number of Distinct Values

Inspection ID ....... 176217
Violations .......... 128236
License # ........... 35223
DBA Name ............ 26224
AKA Name ............ 25049
Address ............. 17617
Latitude ............ 16427
Longitude ........... 16427
Location ............ 16427
Inspection Date ..... 2229
Facility Type ....... 463
Inspection Type ..... 109
Zip ................. 106
City ................ 63
Results ............. 7
Risk ................ 5
State ............... 2


#### Some Functions to Describe the Distinct Values for Each Column and to Identify Redundancy.

In [8]:
def count_column_values(list_dictionaries, column, column_value):# Count of each distinct value in a column.
    count=0
    for dictionary in list_dictionaries:
        if (dictionary[column] == column_value):
            count+=1
    return(count)   

In [9]:
import random

def summarize_distinct_values_for_column(list_dictionaries, column):# Summarises distinct values
    distinct_vals = count_distinct_values_by_column(food_inspection,column)
    print('Distinct Values in', column,' = ',distinct_vals)
    set_values = set_of_distinct_values_by_column(list_dictionaries, column)
    samples = random.sample(set_values,10)
    if distinct_vals > 1000:
        print("\n# Too Many Distinct Values to Provide Report.")      
        print("\nRandom Sample of Values :\n")
        for x in samples:
            print(x)
    else:       
        print('\nValue',spaces("Value"),"Count\n")
        count_and_value = [(count_column_values(food_inspection,column,x),x) for x in set_values]
        count_and_value.sort(key=lambda x: x[0],reverse=True)
        for pair in count_and_value:
            print(pair[1],spaces(pair[1]),pair[0])

In [10]:
def identify_similar_values(list_dictionaries, column, value_a, rate = 0.5):# Identifies the most similar values to a given value
    column_list = list(set_of_distinct_values_by_column(list_dictionaries, column))
    sim_list = []
    value_a_chars = set(value_a)
    value_a_chars_len = len(value_a_chars)
    for value_b in column_list:
        value_b_chars = set(value_b)
        value_b_len = len(value_b)
        value_b_chars_len = len(value_b_chars)
        in_both = len(value_a_chars & value_b_chars)
        in_one = len(value_a_chars ^ value_b_chars)
        similarity = round(in_both/value_a_chars_len,2) 
        if  similarity >= rate and (value_a != value_b):
            sim_list.append((similarity, 1/value_b_len, value_a, value_b))  
    sim_list.sort(reverse=True)
    return sim_list

In [11]:
def print_similar_values(list_dictionaries, column, value_a, rate=0.5):# Prints the most similar values to a specific value
    sim_to_value_a = identify_similar_values(list_dictionaries, column, value_a, rate)
    for x in sim_to_value_a:
        print(x[3],spaces(x[3],dist=40), "contains",int(x[0]*100),"% of the chars in",value_a)

In [12]:
def replace_value(list_of_dictionaries, column, value, replacement): # Replaces values in a column
    cnt = 0
    for row in list_of_dictionaries:
        if row[column] == value:
            if cnt == 0:
                print("Was:",row[column])
            row[column] = replacement
            if cnt == 0:
                print("Now:",row[column])
            cnt+=1
    print(cnt,"Values Changed")   

# CLEANING OPERATIONS

#### Identify five cleaning operations to perform on the raw CSV data. 
For each cleaning operations, clearly articulate the following: 
1. Describe the problem you have identified with the data (i.e., what’s wrong), 
2. Describe the proposed solution to the identified problem (i.e., the fix), 
3. Provide a justification for the proposed solution (i.e., the why), 
4. Present the runable Python code with applies the solution to the raw data, and 
5. Present a before and after sample of the impacted data to show that your code works

 #### Cleaning Operation 1:   
 ##### 1. Problem:    Redundancy in "AKA Name" Column.
 ##### 2. Proposed Solution:    Convert to uppercase (removes some duplicates), Identify certain duplicates and change them.
 ##### 3. Justification:    Upper and lowercase variants are redundant. Some values are clearly miss-spellings, errors or just unnecessary variants.
 ##### 4.  See code below for runnable python solution.
 ##### 4.  See code below for confirmation of all cleaning operations.

##### Summarise The "AKA Name" Column Before Making Changes!

In [13]:
summarize_distinct_values_for_column(food_inspection, "AKA Name")# Summarise Column. WARNING! Too many values. Gives sample only.

Distinct Values in AKA Name  =  25049

# Too Many Distinct Values to Provide Report.

Random Sample of Values :

THREE CHEFS RESTURANT
M. L. K. CONVENIENT STORE
MCDONALD'S  (T3 HK FOOD COURT)
CORONA'S COFFEE SHOP
GARDE MANGER KITCHEN ROOM 330
57TH SUPERMARKET, INC.
FIESTA PARTY TACOS
BREAKAWAY S-101
HAROLD'S CHICKEN SHACK #6
Checkers


##### Convert all values to uppercase. This eradicates 265 duplicate values. Distinct Value Count was 25049 now 24787.

In [14]:
for row in food_inspection:
    row["AKA Name"] = row["AKA Name"].upper()
summarize_distinct_values_for_column(food_inspection, "AKA Name")# Summarise Column

Distinct Values in AKA Name  =  24787

# Too Many Distinct Values to Provide Report.

Random Sample of Values :

SAN LUIS FREEZE
SALOMON SMITH BARNEY EXECUTIVE
PANNA DOLCE
BLEACHER BUMS #4023
WALGREENS #162
FANTASTIC PRODUCE
MOD PIZZA
MCDONALDS #490
PUPUSERIA EL EXCELENTE
LOLLY POP NURSERY SCHOOL INC


##### Find similar values to "MCDONALD'S". For this we use my function print_similar_values()

In [15]:
print_similar_values(food_inspection,"AKA Name","MCDONALD'S", rate = 1)

MC DONALD'S ............................. contains 100 % of the chars in MCDONALD'S
MCDONALD'S II ........................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #490 ......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6337 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6310 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #5813 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #5246 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #26364 ....................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #20104 ....................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #11290 ....................... contains 100 % of the chars in MCDONALD'S
MILADY'S LUNCHROOM ...................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S EXPRESS ...................... contains 100 % of the chars in MCD

##### Replace identified duplicates using standard replace() built-in
##### To change just the "MC DONALD'S" part it is best to use the built-in replace() function

In [16]:
names = ["MC DONALD'S ", "MC DONALD' S","MC DONALD' S","MC DONALD'S","MCDONALD' S"]
for row in food_inspection:
    for name in names:
        row["AKA Name"] = row['AKA Name'].replace(name,"MCDONALD'S")

In [17]:
print_similar_values(food_inspection,"AKA Name","MCDONALD'S", rate = 1)
# We can see values like MC DONALD'S have been changed to the standard "MCDONALD'S"

MCDONALD'S II ........................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #490 ......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6337 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6310 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #5813 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #5246 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #26364 ....................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #20104 ....................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #11290 ....................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S # 5618 ....................... contains 100 % of the chars in MCDONALD'S
MILADY'S LUNCHROOM ...................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S EXPRESS ...................... contains 100 % of the chars in MCD

##### To change specific values i can use my replace_value() function
##### This function is good for replacing values like "MC DONALD'S RESTAURANTS" with just "MCDONALD'S"

In [18]:
replace_value(food_inspection,"AKA Name","MCDONALD'S RESTAURANTS","MCDONALD'S")
print()
replace_value(food_inspection,"AKA Name","MCDONALD'S CORPORATION","MCDONALD'S")
print()
replace_value(food_inspection,"AKA Name","MCDONALD'S RESTAURANT","MCDONALD'S")

Was: MCDONALD'S RESTAURANTS
Now: MCDONALD'S
20 Values Changed

Was: MCDONALD'S CORPORATION
Now: MCDONALD'S
17 Values Changed

Was: MCDONALD'S RESTAURANT
Now: MCDONALD'S
86 Values Changed


 #### Cleaning Operation 2:   
 ##### 1. Problem:    Redundancy in "DBA Name" Column.
 ##### 2. Proposed Solution:    Convert to uppercase (removes some duplicates), Identify certain duplicates and change them.
 ##### 3. Justification:    Upper and lowercase variants are redundant. Some values are clearly miss-spellings, errors or just unnecessary variants.
 ##### 4.  See code below for runnable python solution.
 ##### 4.  See code below for confirmation of all cleaning operations.

##### Summarise "DBA Name" Column Before Making Changes!

In [19]:
summarize_distinct_values_for_column(food_inspection, "DBA Name")# Summarise Column WARNING! Too many values.

Distinct Values in DBA Name  =  26224

# Too Many Distinct Values to Provide Report.

Random Sample of Values :

SUPER SAVE
MONTESSORI ACADEMY OF CHICAGO, 1, LLC
G & G 59TH ST INC
COZY CORNER DINER AND PANCAKE HOUSE
MICHELLA TERRACE
A KARRASEL CHILD CARE CENTERS
INDIO COCINA MEXICANA
JK'S BAR-B-QUE CHICAGO AVE
MFK
NICKY'S GRILL & YOGURT OASIS


#### Convert all "DBA Name" values to Uppercase. This removes 76 redundancies.

In [20]:
for row in food_inspection:
    row["DBA Name"] = row["DBA Name"].upper()

In [21]:
summarize_distinct_values_for_column(food_inspection, "DBA Name")# Summarise Column, Uppercasing removes 76 duplictes.

Distinct Values in DBA Name  =  25948

# Too Many Distinct Values to Provide Report.

Random Sample of Values :

PIZZERIA LA ESTRELLA
WOOD LAWN AMOCO
DOUBLETREE BY HILTON CHICAGO-MAGNIFICENT MILE
TASTE OF CHICAGO
LAESPIGA DE ORO BAKERY
KAFFECCINO
ALTGELD  ANX
PATINO'S GRILL
BEST SUBS
15TH & KEDZIE VARIETY STORE


#### Replace duplicate values for "MCDONALD'S".
#### Identify similar values.

In [22]:
print_similar_values(food_inspection,"DBA Name","MCDONALD'S", rate = 1)

MC DONALD'S ............................. contains 100 % of the chars in MCDONALD'S
MCDONALD'S II ........................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #788 ......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #490 ......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S 26371 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6491 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6337 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6310 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6113 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #5813 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #5246 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #4308 ........................ contains 100 % of the chars in MCD

##### Standard replace() built-in
##### To change just the "MC DONALD'S" part it is better to use the built-in replace() function

In [23]:
names = ["MC DONALD'S ", "MC DONALD' S","MC DONALD' S","MC DONALD'S","MCDONALD' S"]
for row in food_inspection:
    for name in names:
        row["DBA Name"] = row['DBA Name'].replace(name,"MCDONALD'S")

In [24]:
print_similar_values(food_inspection,"DBA Name","MCDONALD'S", rate = 1)

MCDONALD'S II ........................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S6039 .......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S#5697 ......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S#3901 ......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #788 ......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S #490 ......................... contains 100 % of the chars in MCDONALD'S
MCDONALD'S#11559 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S 26371 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6491 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6337 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6310 ........................ contains 100 % of the chars in MCDONALD'S
MCDONALD'S #6113 ........................ contains 100 % of the chars in MCD

##### To change specific values i can use my replace_value() function
##### This function is good for replacing values like "MC DONALD'S RESTAURANTS" with just "MCDONALD'S"

In [25]:
replace_value(food_inspection,"DBA Name","MCDONALD'S RESTAURANTS","MCDONALD'S")
print()
replace_value(food_inspection,"DBA Name","MCDONALD'S CORPORATION","MCDONALD'S")
print()
replace_value(food_inspection,"DBA Name","MCDONALD'S RESTAURANT","MCDONALD'S")
print()
replace_value(food_inspection,"DBA Name","MCDONALDS'S RESTAURANTS","MCDONALD'S")


Was: MCDONALD'S RESTAURANTS
Now: MCDONALD'S
53 Values Changed

Was: MCDONALD'S CORPORATION
Now: MCDONALD'S
31 Values Changed

Was: MCDONALD'S RESTAURANT
Now: MCDONALD'S
111 Values Changed

Was: MCDONALDS'S RESTAURANTS
Now: MCDONALD'S
12 Values Changed


#### Replace duplicate values for "NOODLES AND COMPANY"
#### Identify the values most similar to "NOODLES AND COMPANY"

In [26]:
print_similar_values(food_inspection,"DBA Name","NOODLES AND COMPANY", rate = 1)
# NOODLES & COMPANY -> This one i think should be changed.
# NOODLES & COMPANY #673 -> This one i am not sure about.

NOODLES & COMPANY ....................... contains 100 % of the chars in NOODLES AND COMPANY
ALPHONSUS ACADEMY ....................... contains 100 % of the chars in NOODLES AND COMPANY
SPAULDING/HOPE ACADEMY .................. contains 100 % of the chars in NOODLES AND COMPANY
NOODLES & COMPANY #673 .................. contains 100 % of the chars in NOODLES AND COMPANY
DAVIDSON HOTEL COMPANY .................. contains 100 % of the chars in NOODLES AND COMPANY
NELLY'S FOOD MART CORP. ................. contains 100 % of the chars in NOODLES AND COMPANY
SONNETS ACADEMY-WEST LOOP ............... contains 100 % of the chars in NOODLES AND COMPANY
PULASKI COMMUNITY ACADEMY ............... contains 100 % of the chars in NOODLES AND COMPANY
GOOSE ISLAND BEER COMPANY ............... contains 100 % of the chars in NOODLES AND COMPANY
MIDWEST FUEL & OIL COMPANY .............. contains 100 % of the chars in NOODLES AND COMPANY
KIPP ASCEND PRIMARY SCHOOL .............. contains 100 % of the chars 

In [27]:
replace_value(food_inspection,"DBA Name","NOODLES & COMPANY","NOODLES AND COMPANY")
print()

Was: NOODLES & COMPANY
Now: NOODLES AND COMPANY
79 Values Changed



 #### Cleaning Operation 3:   
 ##### 1. Problem:    Redundancy in "City" Column.
 ##### 2. Proposed Solution:    Convert to uppercase (removes some duplicates), Identify certain duplicates and change them.
 ##### 3. Justification:    Upper and lowercase variants are redundant. Some values are clearly miss-spellings, errors or just unnecessary variants.
 ##### 4.  See code below for runnable python solution.
 ##### 4.  See code below for confirmation of all cleaning operations.

##### Summarise "City" Column Before Making Changes!

In [28]:
summarize_distinct_values_for_column(food_inspection, "City")# Summarise Column

Distinct Values in City  =  63

Value ............... Count

CHICAGO ............. 175423
Chicago ............. 284
 .................... 157
chicago ............. 89
CCHICAGO ............ 44
SCHAUMBURG .......... 23
MAYWOOD ............. 15
ELK GROVE VILLAGE ... 13
CHICAGOCHICAGO ...... 11
CHicago ............. 11
EVANSTON ............ 10
CICERO .............. 9
CHESTNUT STREET ..... 9
SKOKIE .............. 8
INACTIVE ............ 8
CHCHICAGO ........... 6
NILES NILES ......... 6
OAK PARK ............ 5
ELMHURST ............ 5
CALUMET CITY ........ 5
WORTH ............... 5
SUMMIT .............. 4
CHARLES A HAYES ..... 4
CHCICAGO ............ 3
BRIDGEVIEW .......... 3
ALSIP ............... 3
CHICAGOI ............ 3
SCHILLER PARK ....... 3
EAST HAZEL CREST .... 3
PLAINFIELD .......... 3
BLUE ISLAND ......... 2
STREAMWOOD .......... 2
BEDFORD PARK ........ 2
BERWYN .............. 2
312CHICAGO .......... 2
CHICAGO HEIGHTS ..... 2
HIGHLAND PARK ....... 2
BANNOCKBURNDEERFIELD  2
NAPERVILLE

#### Convert all "City" values to Uppercase. This removes 5 redundancies.

In [29]:
for row in food_inspection:
    row["City"] = row["City"].upper()
summarize_distinct_values_for_column(food_inspection, "City")# Summarise Column

Distinct Values in City  =  58

Value ............... Count

CHICAGO ............. 175807
 .................... 157
CCHICAGO ............ 44
SCHAUMBURG .......... 23
MAYWOOD ............. 16
ELK GROVE VILLAGE ... 13
CHICAGOCHICAGO ...... 11
EVANSTON ............ 10
CICERO .............. 9
CHESTNUT STREET ..... 9
SKOKIE .............. 8
INACTIVE ............ 8
CHCHICAGO ........... 6
NILES NILES ......... 6
OAK PARK ............ 5
ELMHURST ............ 5
CALUMET CITY ........ 5
WORTH ............... 5
SUMMIT .............. 4
CHARLES A HAYES ..... 4
ALSIP ............... 4
CHCICAGO ............ 3
BRIDGEVIEW .......... 3
CHICAGOI ............ 3
SCHILLER PARK ....... 3
EAST HAZEL CREST .... 3
PLAINFIELD .......... 3
BLUE ISLAND ......... 2
STREAMWOOD .......... 2
BEDFORD PARK ........ 2
BERWYN .............. 2
312CHICAGO .......... 2
CHICAGO HEIGHTS ..... 2
HIGHLAND PARK ....... 2
BANNOCKBURNDEERFIELD  2
NAPERVILLE .......... 2
BLOOMINGDALE ........ 1
TINLEY PARK ......... 1
COUNTRY CLUB H

#### Identify similar values to "CHICAGO" these may be miss-spellings, errors or unnecessary variants.

In [30]:
print_similar_values(food_inspection,"City", "CHICAGO",0.9)

CHICAGOI ................................ contains 100 % of the chars in CHICAGO
CHICAGO. ................................ contains 100 % of the chars in CHICAGO
CHCICAGO ................................ contains 100 % of the chars in CHICAGO
CCHICAGO ................................ contains 100 % of the chars in CHICAGO
CHCHICAGO ............................... contains 100 % of the chars in CHICAGO
312CHICAGO .............................. contains 100 % of the chars in CHICAGO
CHICAGOHICAGO ........................... contains 100 % of the chars in CHICAGO
CHICAGOCHICAGO .......................... contains 100 % of the chars in CHICAGO
CHICAGO HEIGHTS ......................... contains 100 % of the chars in CHICAGO


#### Replace duplicate values for "CHICAGO"

In [31]:
replace_value(food_inspection,"City","CCHICAGO","CHICAGO")
print()
replace_value(food_inspection,"City","CHICAGOCHICAGO","CHICAGO")
print()
replace_value(food_inspection,"City","CHICAGOHICAGO","CHICAGO")
print()
replace_value(food_inspection,"City","CHCICAGO","CHICAGO")
print()
replace_value(food_inspection,"City","CHICAGO.","CHICAGO")
print()
replace_value(food_inspection,"City","CHICAGOI","CHICAGO")
print()
replace_value(food_inspection,"City","CHCHICAGO","CHICAGO")

Was: CCHICAGO
Now: CHICAGO
44 Values Changed

Was: CHICAGOCHICAGO
Now: CHICAGO
11 Values Changed

Was: CHICAGOHICAGO
Now: CHICAGO
1 Values Changed

Was: CHCICAGO
Now: CHICAGO
3 Values Changed

Was: CHICAGO.
Now: CHICAGO
1 Values Changed

Was: CHICAGOI
Now: CHICAGO
3 Values Changed

Was: CHCHICAGO
Now: CHICAGO
6 Values Changed


 #### Cleaning Operation 4:   
 ##### 1. Problem:    Redundancy in "Inspection Type" column.
 ##### 2. Proposed Solution:    Convert to uppercase (removes some duplicates), Identify certain duplicates and change them.
 ##### 3. Justification:    Upper and lowercase variants are redundant. Some values are clearly miss-spellings, errors or just unnecessary variants.
 ##### 4.  See code below for runnable python solution.
 ##### 4.  See code below for confirmation of all cleaning operations.

##### Summarise "Imspection Type" Column Before Making Changes!

In [32]:
summarize_distinct_values_for_column(food_inspection, "Inspection Type")

Distinct Values in Inspection Type  =  109

Value ............... Count

Canvass ............. 93079
License ............. 22944
Canvass Re-Inspection  18407
Complaint ........... 16317
License Re-Inspection  8141
Complaint Re-Inspection  6713
Short Form Complaint  6318
Suspected Food Poisoning  765
Consultation ........ 669
License-Task Force .. 605
Tag Removal ......... 603
Out of Business ..... 284
Recent Inspection ... 268
Task Force Liquor 1475  254
Suspected Food Poisoning Re-inspection  173
Complaint-Fire ...... 161
Short Form Fire-Complaint  113
Special Events (Festivals)  63
No Entry ............ 60
Package Liquor 1474 . 44
Complaint-Fire Re-inspection  44
OUT OF BUSINESS ..... 22
LICENSE REQUEST ..... 19
Pre-License Consultation  15
Non-Inspection ...... 14
Not Ready ........... 10
NO ENTRY ............ 7
Illegal Operation ... 5
SFP ................. 4
SFP/COMPLAINT ....... 4
no entry ............ 4
SPECIAL TASK FORCE .. 2
LICENSE RENEWAL FOR DAYCARE  2
LICENSE/NOT READY ... 

#### Convert all "Inspection Type" values to Uppercase. This removes 12 redundancies.

In [33]:
for row in food_inspection:
    row["Inspection Type"] = row["Inspection Type"].upper()
summarize_distinct_values_for_column(food_inspection, "Inspection Type")# Summarise Column

Distinct Values in Inspection Type  =  97

Value ............... Count

CANVASS ............. 93080
LICENSE ............. 22946
CANVASS RE-INSPECTION  18407
COMPLAINT ........... 16317
LICENSE RE-INSPECTION  8141
COMPLAINT RE-INSPECTION  6713
SHORT FORM COMPLAINT  6318
SUSPECTED FOOD POISONING  765
CONSULTATION ........ 669
LICENSE-TASK FORCE .. 605
TAG REMOVAL ......... 603
OUT OF BUSINESS ..... 306
RECENT INSPECTION ... 269
TASK FORCE LIQUOR 1475  254
SUSPECTED FOOD POISONING RE-INSPECTION  173
COMPLAINT-FIRE ...... 161
SHORT FORM FIRE-COMPLAINT  113
NO ENTRY ............ 72
SPECIAL EVENTS (FESTIVALS)  63
PACKAGE LIQUOR 1474 . 44
COMPLAINT-FIRE RE-INSPECTION  44
LICENSE REQUEST ..... 19
PRE-LICENSE CONSULTATION  15
NON-INSPECTION ...... 14
NOT READY ........... 10
SFP/COMPLAINT ....... 6
ILLEGAL OPERATION ... 5
SFP ................. 4
SPECIAL TASK FORCE .. 3
LICENSE CONSULTATION  3
LICENSE RENEWAL FOR DAYCARE  2
LICENSE/NOT READY ... 2
REINSPECTION OF 48 HOUR NOTICE  2
TASK FORCE LIQ

#### Identify values similar to CANVASS.

In [34]:
print_similar_values(food_inspection,"Inspection Type","CANVASS", rate = .9)

CANVAS .................................. contains 100 % of the chars in CANVASS
CANVASS FOR RIB FEST .................... contains 100 % of the chars in CANVASS
CANVASS/SPECIAL EVENT ................... contains 100 % of the chars in CANVASS
CANVASS RE-INSPECTION ................... contains 100 % of the chars in CANVASS
CANVASS SPECIAL EVENTS .................. contains 100 % of the chars in CANVASS
SPECIAL EVENTS (FESTIVALS) .............. contains 100 % of the chars in CANVASS
CANVASS SCHOOL/SPECIAL EVENT ............ contains 100 % of the chars in CANVASS
KITCHEN CLOSED FOR RENOVATION ........... contains 100 % of the chars in CANVASS
TASK FORCE(1470) LIQUOR TAVERN .......... contains 100 % of the chars in CANVASS
CANVASS RE INSPECTION OF CLOSE UP ....... contains 100 % of the chars in CANVASS


#### "CANVAS" changed to the more common CANVASS.

In [35]:
replace_value(food_inspection,"Inspection Type","CANVAS","CANVASS")

Was: CANVAS
Now: CANVASS
1 Values Changed


#### "CANVASS SPECIAL EVENTS" changed to the more common "CANVASS/SPECIAL EVENT".

In [36]:
replace_value(food_inspection,"Inspection Type","CANVASS SPECIAL EVENTS","CANVASS/SPECIAL EVENT")

Was: CANVASS SPECIAL EVENTS
Now: CANVASS/SPECIAL EVENT
1 Values Changed


#### Identify values similar to "FIRE COMPLAINT". Redundancy in values "COMPLAINT-FIRE"

In [37]:
print_similar_values(food_inspection,"Inspection Type","FIRE COMPLAINT", rate = .9)

SHORT FORM FIRE-COMPLAINT ............... contains 100 % of the chars in FIRE COMPLAINT
COMPLAINT-FIRE RE-INSPECTION ............ contains 100 % of the chars in FIRE COMPLAINT
FINISH COMPLAINT INSPECTION FROM 5-18-10  contains 100 % of the chars in FIRE COMPLAINT
COMPLAINT-FIRE .......................... contains 92 % of the chars in FIRE COMPLAINT
SHORT FORM COMPLAINT .................... contains 92 % of the chars in FIRE COMPLAINT
COMPLAINT RE-INSPECTION ................. contains 92 % of the chars in FIRE COMPLAINT
NO ENTRY-SHORT COMPLAINT) ............... contains 92 % of the chars in FIRE COMPLAINT
CLOSE-UP/COMPLAINT REINSPECTION ......... contains 92 % of the chars in FIRE COMPLAINT
TASK FORCE LIQUOR INSPECTION 1474 ....... contains 92 % of the chars in FIRE COMPLAINT
CANVASS RE INSPECTION OF CLOSE UP ....... contains 92 % of the chars in FIRE COMPLAINT
LICENSE RENEWAL INSPECTION FOR DAYCARE .. contains 92 % of the chars in FIRE COMPLAINT
LICENSE TASK FORCE / NOT -FOR-PROFIT CLU

##### Replace duplicate "COMPLAINT-FIRE" values with "FIRE COMPLAINT"

In [38]:
replace_value(food_inspection,"Inspection Type","COMPLAINT-FIRE","FIRE COMPLAINT")

Was: COMPLAINT-FIRE
Now: FIRE COMPLAINT
161 Values Changed


#### Identify values similar to "LICENSE TASK FORCE / NOT -FOR-PROFIT CLUB"
#### Data entry error: "LICENSE TASK FORCE / NOT -FOR-PROFIT CLU" 

In [39]:
print_similar_values(food_inspection,"Inspection Type","LICENSE TASK FORCE / NOT -FOR-PROFIT CLUB", rate = .9)

LICENSE TASK FORCE / NOT -FOR-PROFIT CLU  contains 94 % of the chars in LICENSE TASK FORCE / NOT -FOR-PROFIT CLUB


In [40]:
replace_value(food_inspection,"Inspection Type","LICENSE TASK FORCE / NOT -FOR-PROFIT CLU","LICENSE TASK FORCE / NOT -FOR-PROFIT CLUB")

Was: LICENSE TASK FORCE / NOT -FOR-PROFIT CLU
Now: LICENSE TASK FORCE / NOT -FOR-PROFIT CLUB
1 Values Changed


 #### Cleaning Operation 5:   
 ##### 1. Problem:    Redundancy in "Facility type" column.
 ##### 2. Proposed Solution:    Convert to uppercase (removes some duplicates), Identify certain duplicates and change them.
 ##### 3. Justification:    Upper and lowercase variants are redundant. Some values are clearly miss-spellings, errors or just unnecessary variants.
 ##### 4.  See code below for runnable python solution.
 ##### 4.  See code below for confirmation of all cleaning operations.

##### Summarise The "Facility Type" Column Before Making Changes!

In [41]:
summarize_distinct_values_for_column(food_inspection, "Facility Type")

Distinct Values in Facility Type  =  463

Value ............... Count

Restaurant .......... 117049
Grocery Store ....... 23051
School .............. 10914
 .................... 4703
Bakery .............. 2588
Daycare (2 - 6 Years)  2563
Children's Services Facility  2303
Daycare Above and Under 2 Years  1981
Long Term Care ...... 1122
Catering ............ 1034
Mobile Food Dispenser  832
Liquor .............. 791
Daycare Combo 1586 .. 767
Mobile Food Preparer  526
Wholesale ........... 526
Golden Diner ........ 484
Hospital ............ 464
TAVERN .............. 241
Daycare (Under 2 Years)  215
Special Event ....... 201
Shared Kitchen User (Long Term)  187
BANQUET HALL ........ 130
GAS STATION ......... 109
KIOSK ............... 104
Shelter ............. 90
Mobile Prepared Food Vendor  84
Shared Kitchen ...... 80
LIVE POULTRY ........ 55
Navy Pier Kiosk ..... 54
DAYCARE ............. 53
GROCERY/RESTAURANT .. 50
BANQUET ............. 49
Daycare (2 Years) ... 43
STADIUM ............. 41

Lounge .............. 2
LIQOUR BREWERY TASTING  2
HOT DOG CART ........ 2
VFW HALL ............ 2
TEA STORE ........... 2
ART GALLERY W/WINE AND BEER  2
smoothie bar ........ 2
ASSISSTED LIVING .... 2
candy/gelato ........ 2
HERBAL DRINKS ....... 2
GROCERY/ RESTAURANT . 2
URBAN FARM .......... 2
CONVNIENCE STORE .... 2
unlicensed facility . 2
MASSAGE BAR ......... 2
PREP INSIDE SCHOOL .. 2
Dollar store ........ 2
RETAIL WINE/WINE BAR  2
GAS STATION /SUBWAY MINI MART.  2
UNUSED STORAGE ...... 2
POPCORN CORN ........ 2
1584-DAY CARE ABOVE 2 YEARS  2
DISTRIBUTOR ......... 2
RESTAURANT/LIQUOR ... 2
after school program  2
CONVENIENT STORE .... 2
BUTCHER SHOP ........ 2
CHARTER SCHOOL/CAFETERIA  2
LIQUORE STORE/BAR ... 2
GAS STATION STORE ... 2
KITCHEN DEMO ........ 2
School Cafeteria .... 2
Museum/Gallery ...... 2
LINITED BUSINESS .... 2
MAIN KITCHEN ........ 2
SOUP KITCHEN ........ 2
POPCORN SHOP ........ 2
CLOTHING STORE ...... 2
CAT/LIQUOR .......... 1
MOBILE FROZEN DESSERTS DISPENSER-N

#### Convert all "Facility Type" values to Uppercase. This removes 46 redundancies.

In [42]:
for row in food_inspection:
    row["Facility Type"] = row["Facility Type"].upper()
summarize_distinct_values_for_column(food_inspection, "Facility Type")# Summarise Column

Distinct Values in Facility Type  =  417

Value ............... Count

RESTAURANT .......... 117049
GROCERY STORE ....... 23051
SCHOOL .............. 10921
 .................... 4703
BAKERY .............. 2588
DAYCARE (2 - 6 YEARS)  2563
CHILDREN'S SERVICES FACILITY  2303
DAYCARE ABOVE AND UNDER 2 YEARS  1981
LONG TERM CARE ...... 1122
CATERING ............ 1034
MOBILE FOOD DISPENSER  832
LIQUOR .............. 791
DAYCARE COMBO 1586 .. 767
MOBILE FOOD PREPARER  526
WHOLESALE ........... 526
GOLDEN DINER ........ 484
HOSPITAL ............ 464
TAVERN .............. 278
DAYCARE (UNDER 2 YEARS)  215
SPECIAL EVENT ....... 201
SHARED KITCHEN USER (LONG TERM)  187
BANQUET HALL ........ 149
GAS STATION ......... 128
KIOSK ............... 104
SHELTER ............. 90
MOBILE PREPARED FOOD VENDOR  84
SHARED KITCHEN ...... 80
LIVE POULTRY ........ 70
CONVENIENCE STORE ... 61
BANQUET ............. 59
NAVY PIER KIOSK ..... 54
DAYCARE ............. 53
CAFETERIA ........... 53
CONVENIENCE ......... 52

HERBAL STORE ........ 1
GROCERY/LIQUOR ...... 1
GIFT SHOP ........... 1
(CONVENIENCE STORE) . 1
HERBAL .............. 1
MOBILE DESSERT CART . 1
GIFT/CARD SHOP WITH CANDY  1
TAVERN-LIQUOR ....... 1
HERBALIFE STORE ..... 1
ALTERNATIVE SCHOOL .. 1
SERVICE GAS STATION . 1
WHOLESALE BAKERY .... 1
CHICAGO PARK DISTRICT  1
SHARED KITCHEN USER (LONG TREM)  1
PROTEIN SHAKE BAR ... 1
SERVICE BAR/THEATRE . 1
CHURCH/AFTER SCHOOL PROGRAM  1
VENDING MACHINE ..... 1
CONVENIENCE/GAS STATION  1
NAIL SHOP ........... 1
NON-FOR PROFIT BASEMENT KIT  1
DAY CARE COMBO (1586)  1
MOBILE FROZEN DESSERTS DISPENSER-NON- MOTORIZED  1
MOBILE FOOD DESSERTS VENDOR  1
GROCERY/LIQUOR STORE  1
COMMIASARY .......... 1
INTERNET CAFE ....... 1
MEAT MARKET ......... 1
GROCERY/TAVERN ...... 1
COFFEE ROASTER ...... 1
INCUBATOR ........... 1
NP-KIOSK ............ 1
TAVERN/LIQUOR ....... 1
TAP ROOM/TAVERN/LIQUOR STORE  1
MOBIL FOOD 1315 ..... 1
ART GALLERY ......... 1
MOBILE DESSERT VENDOR  1
DRUG TREATMENT FACILITY  1
WHOLESA

#### Identify & replace similar values to "GROCERY & RESTAURANT"

In [ ]:
print_similar_values(food_inspection,"Facility Type","GROCERY & RESTAURANT", rate = .9)

GROCERY& RESTAURANT ..................... contains 100 % of the chars in GROCERY & RESTAURANT
GROCERY/ RESTAURANT ..................... contains 92 % of the chars in GROCERY & RESTAURANT
RESTAURANT/GROCERY STORE ................ contains 92 % of the chars in GROCERY & RESTAURANT
GROCERY STORE/ RESTAURANT ............... contains 92 % of the chars in GROCERY & RESTAURANT
SUPPORTIVE LIVING FACILITY .............. contains 92 % of the chars in GROCERY & RESTAURANT


In [ ]:
replace_value(food_inspection,"Facility Type","GROCERY STORE/ RESTAURANT","GROCERY & RESTAURANT")
print()
replace_value(food_inspection,"Facility Type","RESTAURANT/GROCERY STORE","GROCERY & RESTAURANT")
print()
replace_value(food_inspection,"Facility Type","GROCERY/ RESTAURANT","GROCERY & RESTAURANT")
print()
replace_value(food_inspection,"Facility Type","GROCERY& RESTAURANT","GROCERY & RESTAURANT")

Was: GROCERY STORE/ RESTAURANT
Now: GROCERY & RESTAURANT
9 Values Changed

Was: RESTAURANT/GROCERY STORE
Now: GROCERY & RESTAURANT
24 Values Changed

Was: GROCERY/ RESTAURANT
Now: GROCERY & RESTAURANT
2 Values Changed

Was: GROCERY& RESTAURANT
Now: GROCERY & RESTAURANT
11 Values Changed


### Store and Export Cleaned Data

In [ ]:
keys = food_inspection[0].keys()
with open('Food-Inspections-cleaned.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys, delimiter=',',lineterminator='\n')
    dict_writer.writeheader()
    dict_writer.writerows(food_inspection)